In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import numpy as np
import pandas as pd
import time
from scipy.special import digamma
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support as score

class matrix_factorization():
	
	def __init__(self,data,orig_data,features,R,alpha,beta):
		self.data = data
		self.orig_data = orig_data
		self.features = features
		self.R = R
		self.alpha = alpha
		self.beta = beta
		self.user_count = data.shape[0]
		self.item_count = data.shape[1]
		self.user_features = np.random.uniform(low=0.1,high=0.9,size=(self.user_count,self.features))
		self.item_features = np.random.uniform(low=0.1,high=0.9,size=(self.features,self.item_count))
		self.gamma = np.random.uniform(low=0.1,high=0.9,size=(self.user_count,self.features))
		self.e_plus = np.random.uniform(low=0.1,high=0.9,size=(self.item_count,self.features))
		self.e_minus = np.random.uniform(low=0.1,high=0.9,size=(self.item_count,self.features))
		self.lambda_ = np.zeros(shape=(self.user_count,self.item_count,self.features))
	
	def MSE(self):
		"""
		Mean squared error function comparing dot product of user-feature row and feature-item column to user-item cell
		"""
		# print("mse")
		# print("user features:\n", self.user_features)
		# print("item features:\n", self.item_features)
		matrix_product = np.matmul(self.user_features,self.item_features)
		print("predicted rating matrix\n", matrix_product)

		scaled_matrix = np.zeros_like(matrix_product)

		scaled_matrix[(matrix_product >= 0.0) & (matrix_product < 0.2)] = 1
		scaled_matrix[(matrix_product >= 0.2) & (matrix_product < 0.4)] = 2
		scaled_matrix[(matrix_product >= 0.4) & (matrix_product < 0.6)] = 3
		scaled_matrix[(matrix_product >= 0.6) & (matrix_product < 0.8)] = 4
		scaled_matrix[matrix_product >= 0.8 ] = 5

		print("predicted rating matrix after scaling\n", scaled_matrix)
		print("actual rating matrix \n", self.orig_data)
	  
		cnt = 0
		mae = 0
		cmae = 0
		rmse = 0
		cmae_cnt = 0
		for u in range(0,self.user_count):
			for i in range(0,self.item_count):
				if self.orig_data[u,i] != 0:
					cnt += 1
					mae += abs(self.orig_data[u,i] - scaled_matrix[u,i])
					rmse += (self.orig_data[u,i] - scaled_matrix[u,i])**2
					if self.orig_data[u,i] >=4 or scaled_matrix[u,i]>=4:
						cmae_cnt += 1
						cmae += abs(self.orig_data[u,i] - scaled_matrix[u,i])
		mae /= cnt
		cmae /= cmae_cnt
		rmse /= cnt
		rmse = rmse ** 0.5
		print("CMAE = ", cmae)
		print("MAE = ", mae)
		print("RMSE = ", rmse)

		# precision, recall, fscore, support = score(self.orig_data, scaled_matrix)
		# print('precision = {}'.format(precision))
		# print('recall = {}'.format(recall))
		# print('fscore = {}'.format(fscore))
		# print('support = {}'.format(support))

		return mae

		# return np.sum(abs(self.orig_data - scaled_matrix))/(self.user_count*self.item_count)
	
	def get_user_item_features(self):
		# print("features")
		for i in range(0,self.user_count):
			self.user_features[i] = self.gamma[i]/np.sum(self.gamma[i])
		for k in range(0,self.features):
			for i in range(0,self.item_count):
				self.item_features[k,i] = self.e_plus[i,k]/(self.e_plus[i,k]+self.e_minus[i,k])

	def update_epsilion(self):
		"""
		Updates every epsilion parameter according to supplied learning rate
		"""   
		# print("epsilion")   
		for i in range(0,self.item_count):
			for j in range(0,self.features):
				val1 = 0
				val2 = 0
				for k in range(0,self.user_count):
					if self.data[k,i] != 0:
						val1 += self.lambda_[k,i,j]*self.R*self.data[k,i]
						val2 += self.lambda_[k,i,j]*self.R*(1-self.data[k,i])
				self.e_plus[i,j] = self.beta + val1
				self.e_minus[i,j] = self.beta + val2

	def update_gamma(self):
		"""
		Updates every gamma parameter according to supplied learning rate
		"""      
		# print("gamma")
		for i in range(0,self.user_count):
			for j in range(0,self.features):
				val = 0
				for k in range(0,self.item_count):
					if self.data[i,k] != 0:
						val += self.lambda_[i,k,j]
				self.gamma[i,j] = self.alpha + val

	def update_lambda(self):
		"""
		Updates every lambda parameter according to supplied learning rate
		"""      
		# print("lambda")
		for u in range(0,self.user_count):
			for i in range(0,self.item_count):
				if self.data[u,i] != 0:
					lambda_dash = np.zeros(self.features)
					for k in range(0,self.features):
						lambda_dash[k] = np.exp(digamma(self.gamma[u,k]) + \
							self.R*self.data[u,i]*digamma(self.e_plus[i,k]) + \
							self.R*(1-self.data[u,i])*digamma(self.e_minus[i,k]) - \
							self.R*digamma(self.e_plus[i,k]+self.e_minus[i,k]))
					for k in range(0,self.features):
						self.lambda_[u,i,k] = lambda_dash[k]/np.sum(lambda_dash)
				
	def train_model(self,iterations=20):
		"""
		Trains model
		"""    
		avg_tim = 0        
		for i in range(iterations):
			start = time.process_time()
			self.update_lambda()
			self.update_gamma()
			self.update_epsilion()
			tim = (time.process_time() - start)
			avg_tim += tim
			print("Time taken by ",(i+1), " iteration = ", tim)
		self.get_user_item_features()
		mse = self.MSE()
		avg_tim /= iterations
		print("Average time taken = ", avg_tim)
	
# configure file path
data_path = 'gdrive/MyDrive/ml-latest-small/'
movies_filename = 'movies.csv'
ratings_filename = 'ratings.csv'
# read data
df_movies = pd.read_csv(
    data_path + movies_filename,
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv(
    (data_path + ratings_filename),
    usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

# pivot ratings into movie features
df_movie_features = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

print(df_movie_features.head(10))
data = df_movie_features.to_numpy()
print(data)

# split data into train and test
train, test = train_test_split(data, test_size=0.2)	

min_max_scaler = preprocessing.MinMaxScaler()
train_n = min_max_scaler.fit_transform(train)
print(train_n)
test_n = min_max_scaler.fit_transform(test)

obj = matrix_factorization(train_n, train, 6, 4, 0.8, 5)
obj.train_model()	
				

movieId  1       2       3       4       ...  193583  193585  193587  193609
userId                                   ...                                
1           4.0     0.0     4.0     0.0  ...     0.0     0.0     0.0     0.0
2           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5           4.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
6           0.0     4.0     5.0     3.0  ...     0.0     0.0     0.0     0.0
7           4.5     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
8           0.0     4.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
9           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
10          0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[10 rows x 9724 columns]
[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0